# Amazing RL Agent

In [19]:
import gymnasium as gym
import torch
import numpy as np
import importlib

# Render screen, set to false if training
RENDER_AGENT = True


In [20]:
# Test pytorch is working

x = torch.rand(size=(1000,))
x = x + 1

# If you have a nvidia gpu with torch installed correctly
# you can move the tensors to the gpu for some extra
# speed, not necessary although.
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Using {DEVICE}")
# When creating a new tensor make sure you move it to a
# device using .to(DEVICE), such as:
x = x.to(DEVICE)
# The tensor in x has been moved to the device


Using cpu


In [21]:
# Import agent
import agents

# Reload module as jupyter notebooks wont update to any changes youve made
importlib.reload(
    agents
)
#agent = agents.Agent()

# 24 + 4 as the controller state is included
agent = agents.DDPG(24, 4)

In [22]:
env = gym.make("BipedalWalker-v3", hardcore=False) # , render_mode="human"
observation, info = env.reset(seed=27)
steps_per_episode = 200 # T
for episode in range(1000): # M

    action = [0, 0, 0, 0]
    rewards = 0
    step = 0
    while(step < steps_per_episode):
        action = agent.choose_action(
            env.action_space, observation 
        )  # this is where you would insert your policy
        observation, reward, terminated, truncated, info = env.step(action)
        rewards += reward
        
        agent.update(observation, reward, terminated, truncated)
        if terminated or truncated:
            observation, info = env.reset()
            action = [0, 0, 0, 0]
        step += 1
        #env.render()
    print(episode, rewards)
    observation, info = env.reset()
    #control.reset()
env.close()


0 tensor(-101.6741, dtype=torch.float64)
1 tensor(-119.0715, dtype=torch.float64)
2 tensor(1.2054, dtype=torch.float64)
3 tensor(-187.4070, dtype=torch.float64)
4 tensor(3.2609, dtype=torch.float64)
5 tensor(-93.9592, dtype=torch.float64)
6 tensor(-106.1744, dtype=torch.float64)
7 tensor(-231.5697, dtype=torch.float64)
8 tensor(-124.2457, dtype=torch.float64)
9 tensor(-89.5002, dtype=torch.float64)
10 tensor(1.6002, dtype=torch.float64)
11 tensor(-100.6612, dtype=torch.float64)
12 tensor(-120.7147, dtype=torch.float64)
13 tensor(-105.2241, dtype=torch.float64)
14 tensor(-205.7133, dtype=torch.float64)
15 tensor(-126.5262, dtype=torch.float64)
16 tensor(-128.2642, dtype=torch.float64)
17 tensor(4.7924, dtype=torch.float64)
18 tensor(-2.1914, dtype=torch.float64)
19 tensor(-117.1950, dtype=torch.float64)
20 tensor(-123.9068, dtype=torch.float64)
21 tensor(-1.9646, dtype=torch.float64)
22 tensor(-94.9807, dtype=torch.float64)
23 tensor(-210.3417, dtype=torch.float64)
24 tensor(4.4389, dty

KeyboardInterrupt: 

In [54]:
# Import agent
import agents

# Reload module as jupyter notebooks wont update to any changes youve made
importlib.reload(
    agents
)
#agent = agents.Agent()

# 24 + 4 as the controller state is included
agent = agents.DDPG(4, 1)
scores = []

In [55]:


env = gym.make('CartPole-v1') # , render_mode="human"
observation, info = env.reset(seed=27)
steps_per_episode = 200 # T
for episode in range(10000): # M
    action = [0]
    rewards = 0
    step = 0
    while(step < steps_per_episode):
        action = agent.choose_action(
            env.action_space, observation 
        )  # this is where you would insert your policy
        observation, reward, terminated, truncated, info = env.step(int(action))
        rewards += reward
        
        agent.update(observation, reward, terminated, truncated)
        if terminated or truncated:
            break
            observation, info = env.reset()
            action = [0]
        step += 1
        #env.render()
    print(episode, rewards)
    scores.append([observation, info])
    observation, info = env.reset()
    #control.reset()
env.close()


AssertionError: tensor([0.]) (<class 'torch.Tensor'>) invalid